In [423]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd

In [424]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [425]:
song_meta = pd.read_json('../res/song_meta.json', typ = 'frame')

In [426]:
song_meta_indexed = song_meta.set_index('id')

In [427]:
train = pd.read_json('../res/train.json', typ = 'frame')

In [428]:
def songs_to_words(songs):
    word = []
    for song in songs:
        word.append(str(song))
    return word

In [429]:
train['song_words'] = train['songs'].apply(songs_to_words)

In [430]:
sentences =  train['song_words'].tolist() + song_album_df_enough['song_word'].tolist()

In [470]:
model = Word2Vec(sentences, min_count=1, size=200, window=200)

In [471]:
model.save('song2vec.model')

In [486]:
result = model.wv.most_similar(str(130275), topn=10)

In [487]:
result

[('181369', 0.9454047679901123),
 ('87907', 0.9325739145278931),
 ('559102', 0.9243788719177246),
 ('516586', 0.9203025698661804),
 ('167779', 0.9142670035362244),
 ('8046', 0.9087990522384644),
 ('392954', 0.9064935445785522),
 ('542063', 0.9041101932525635),
 ('48595', 0.9020164012908936),
 ('30792', 0.9017906785011292)]

In [488]:
result_df = pd.DataFrame(result, columns=['song', 'dist'])

In [489]:
result_df['song_id'] = result_df['song'].apply(lambda x : int(x))

In [490]:
song_meta_indexed.loc[result_df['song_id']]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
id,,,,,,,,
181369,"[GN0303, GN0301]",20130701,Luckynumbers,2190970,[108358],"거품 안 넘치게 따라줘 (Life Is Good) (Feat. Crush, DJ F...",[GN0300],[다이나믹 듀오]
87907,"[GN0302, GN0301]",20130311,Fantasy,2177662,[169115],It`s Over (Feat. Swings),[GN0300],[진보]
559102,"[GN0303, GN0301]",20140716,Up All Night,2267501,[532029],Up All Night (Feat. Mayson The Soul),[GN0300],[빈지노 (Beenzino)]
516586,[GN0401],20120207,New Breed,2076266,[443420],New Breed (Intro),[GN0400],[박재범]
167779,"[GN1205, GN1208, GN1201]",20030407,Blueprint 2.1,341652,[16320],Excuse Me Miss,[GN1200],[JAY-Z]
8046,"[GN1501, GN1510, GN1506]",20070322,DJMAX Portable 2 - Vocal Paradigm 2,348215,[219649],Stalker,[GN1500],[ND Lee]
392954,"[GN0303, GN0301]",20130409,Red Light,2180913,[537920],Click Me (2013) (Feat. Dok2),[GN0300],[Zion.T]
542063,"[GN0401, GN0402]",20130311,Fantasy,2177662,[169115],Fantasy,[GN0400],[진보]
48595,"[GN1208, GN1201]",20061031,Public Warning,346906,[217677],Those Were The Days,[GN1200],[Lady Sovereign]


In [491]:
def get_reco_song_w2v(song_list, ignore_list=[], size=100):
    reco_num_per_song = 100
    reco_list = []
    
    for idx, song in enumerate(song_list):
        if str(song) in model.wv.vocab:
            reco_list.extend(model.wv.most_similar(str(song), topn=10))
    #print(reco_list)
        
    reco_df = pd.DataFrame(reco_list, columns=['song', 'dist'])
    reco_count_df = reco_df.groupby('song')['dist'].sum().sort_values(ascending=False).reset_index()
    reco_count_df['song_id'] = reco_count_df['song'].apply(lambda x : int(x))
    reco_top = reco_count_df[~reco_count_df['song_id'].isin(ignore_list)].head(size)
    
    print(reco_top.head())
    return reco_top['song_id'].tolist()

In [492]:
def reco_song_w2v(song_list, ignore_list=[], size=100):
    if len(song_list) == 0:
        return []
    reco_list = get_reco_song_w2v(song_list, ignore_list, size)
    #print(reco_list)
    return reco_list

In [493]:
song_index = 1000

In [494]:
val.iloc[song_index]

tags                                                           []
id                                                          92054
plylst_title                                                     
songs           [529443, 295827, 691965, 641745, 587193, 28024...
like_cnt                                                        5
updt_date                                 2017-10-26 18:08:30.000
Name: 1000, dtype: object

In [495]:
reco_result = reco_song_w2v(val.iloc[song_index]['songs'])

     song      dist  song_id
0  184540  3.954418   184540
1  366797  3.929828   366797
2   42009  3.869028    42009
3  356571  3.784903   356571
4  565582  3.771737   565582


In [496]:
len(set(reco_result))

100

In [497]:
len(set(reco_result) & set(val.iloc[song_index]['songs']))

13

In [498]:
song_meta_indexed.loc[val.iloc[song_index]['songs']]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
id,,,,,,,,
529443,"[GN0302, GN0301]",20130729,쇼미더머니2 Part.3,2194706,[606622],"Like That (Feat. 주희 Of 에이트, 바스커션)",[GN0300],[지조]
295827,"[GN0303, GN0301]",20140806,쇼미더머니3 올티vs육지담,2272303,[669972],OLL` Ready,[GN0300],[올티 (Olltii)]
691965,"[GN2502, GN2501, GN0303, GN2504, GN0301]",20140808,쇼미더머니3 Part.1,2273085,[742760],BE I,"[GN2500, GN0300]",[B.I]
641745,"[GN0303, GN0301]",20150801,쇼미더머니 4 Episode 2,2332701,"[727519, 716177, 241227]","MY ZONE (Feat. 산이, 버벌진트)",[GN0300],"[블랙넛 (Black Nut), 마이크로닷 (Microdot), 베이식 (Basick)]"
587193,"[GN0303, GN0301]",20160618,쇼미더머니 5 Episode 1,2692133,"[219621, 618913, 340668, 838923]",무궁화,[GN0300],"[매드클라운, 도넛맨 (Donutman), 보이비, #Gun]"
28024,"[GN0303, GN0301]",20160625,쇼미더머니 5 Episode 2,2693749,"[219621, 838923]",비행소년 (Feat. 거미),[GN0300],"[매드클라운, #Gun]"
226272,"[GN0303, GN0301]",20150815,쇼미더머니 4 Episode 4,2334942,[241227],GXNZI (Feat. 바스코),[GN0300],[베이식 (Basick)]
354977,"[GN0303, GN0301]",20150815,쇼미더머니 4 Episode 4,2334942,[230010],MORE THAN A TV STAR (Feat. 이하이),[GN0300],[이노베이터]
702476,"[GN0303, GN0301]",20150815,쇼미더머니 4 Episode 4,2334942,"[727519, 6985, 402526]",M.I.L.E. (Make It Look Easy),[GN0300],"[블랙넛 (Black Nut), 버벌진트, San E]"


In [499]:
song_meta_indexed.loc[reco_result].head(30)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket
id,,,,,,,,
184540,"[GN0303, GN0301]",20160716,쇼미더머니 5 Episode 5,2698200,"[27744, 314367, 695757]",공중도덕 part 2 (Air DoTheQ part 2),[GN0300],"[The Quiett, Dok2, 수퍼비]"
366797,"[GN0303, GN0301]",20160716,쇼미더머니 5 Episode 5,2698200,[729560],MM,[GN0300],[C JAMM]
42009,"[GN0303, GN0301]",20150905,쇼미더머니4 Episode 7,2638755,"[241227, 6985, 402526]",I`m The Man,[GN0300],"[베이식 (Basick), 버벌진트, San E]"
356571,"[GN0303, GN0301]",20150801,쇼미더머니 4 Episode 2,2332701,"[743761, 340667, 742773]",거북선 (Feat. 팔로알토),[GN0300],"[Ja Mezz, Andup (앤덥), MINO (송민호)]"
565582,"[GN0303, GN0301]",20170805,쇼미더머니 6 Episode 1,10085618,"[743761, 944760, 1866691, 782332]","도박 (Life Is a Gamble) (Feat. 박재범, Dok2)",[GN0300],"[Ja Mezz, ness (네스), Woodie Gochild, Junoflo (..."
429174,"[GN0303, GN0301]",20170701,"공중도덕 3 (Feat. Dok2, myunDo, Keem Hyoeun, The Q...",10075975,[695757],"공중도덕 3 (Feat. Dok2, myunDo, Keem Hyoeun, The Q...",[GN0300],[수퍼비]
474414,"[GN0303, GN0301]",20170323,Look At It,10048271,"[695757, 926455]",Look At It (Prod. By N-Soul),[GN0300],"[수퍼비, myunDo]"
503911,"[GN0303, GN0301]",20160716,쇼미더머니 5 Episode 5,2698200,"[739740, 428847]",쌈박자 (XamBaqJa) (Prod. by BewhY),[GN0300],"[BewhY (비와이), 사이먼 도미닉]"
88623,"[GN0303, GN0301]",20160716,쇼미더머니 5 Episode 5,2698200,[219621],나쁜 피 (Bad Blood),[GN0300],[매드클라운]
